<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import numpy as np
import os
import re
from copy import deepcopy
from tqdm import tqdm

In [2]:
train_data = pd.read_csv("./train.csv")
test_data = pd.read_csv("./sample_submission.csv")

In [3]:
title_ids = list(set(train_data["Id"]))
id_title_dict = dict()
for tid in title_ids:
    id_title_dict[tid] = pd.read_json(os.path.join("./train/", tid+".json"))

In [4]:
id_datalabel_dict = dict()
for id_, dataset_ in zip(train_data["Id"], train_data["dataset_label"]):
    id_datalabel_dict.setdefault(id_, []).append(dataset_)

In [5]:
def datalabels_match_sen(data_labels, sen):
    for datalabel_ in data_labels:
        if sen.find(datalabel_):
            return True
    
    return False

In [6]:
dfs = []

for tid in tqdm(title_ids):
    data_labels = id_datalabel_dict[tid]
    
    sen_res = []
    encoding_res = []
    labels = []
    dataset_labels = []
    for text in id_title_dict[tid]["text"]:
        sentences = re.split('[.!?]', text)
        for sen in sentences:
            sen_res.append(deepcopy(sen))
            encodings = ["O"] * len(sen.split(" "))
#             print(encodings)
#             print(sen)
#             print("====")
            
            for data_label_ in data_labels:
                flag = 0
                str_start = sen.find(data_label_)
                
                if str_start > -1:
#                     print(sen, data_label_)
                    sen = list(sen)
#                     print(sen)
                    for idx in range(str_start, str_start+len(data_label_)): 
                        sen[idx] = "¥" if sen[idx] != " " else " "
                    sen = "".join(sen)
#                     print(sen)
                    words = sen.split(" ")
                    encodings = []
                    
                    
                    for word in words:
                        if word.find("¥") > -1:
                            flag += 1
                            if flag == 1:
                                encodings.append("B-DS")
                            else:
                                encodings.append("I-DS")
                        else:
                            if flag > 0:
                                encodings[-1] = "E-DS"
                                flag = 0
                            encodings.append("O")
                    break
                    
            encoding_res.append(encodings)
            labels.append(1 if str_start > -1 else 0)
            dataset_labels.append(data_label_ if str_start > -1 else "Null")
            
    df = pd.DataFrame()
    df["Sentence"] = sen_res
    df["Mask"] = encoding_res
    df["Id"] = tid
    df["Label"] = labels
    df["Dataset_label"] = dataset_labels
    
    dfs.append(df)



100%|██████████| 14316/14316 [01:34<00:00, 151.71it/s]


In [7]:
dfs = pd.concat(dfs, axis=0)
dfs = dfs[dfs["Sentence"] != ""]
dfs.to_csv("data_preprocessed2.csv")

In [9]:
dfs[dfs["Label"]==1]

,Sentence,Mask,Id,Label,Dataset_label
2,"In this study, we examined Turkish teachers' ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",401513e7-49eb-4ee2-ae2a-790416de4600,1,Trends in International Mathematics and Scienc...
62,"\nIn this context, the purpose of the current ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",401513e7-49eb-4ee2-ae2a-790416de4600,1,Trends in International Mathematics and Scienc...
25,"Satellite sea surface temperature, hydrograph...","[O, O, O, O, O, O, O, O, B-DS, I-DS, E-DS, O, ...",92064981-12b3-45e2-92dc-c195bff04495,1,World Ocean Database
2,In this policy brief we analyze data from 37 ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",a7fc8a6f-40ea-46db-ab72-a49007d088ae,1,Trends in International Mathematics and Scienc...
158,In studying this phenomenon in the nationally...,"[O, O, O, O, O, O, O, O, O, B-DS, I-DS, I-DS, ...",8468ffc2-b5bb-4e91-af1d-0b874a889158,1,Early Childhood Longitudinal Study
...,...,...,...,...,...
84,In a subset of ADNI 1 participants with avail...,"[O, O, O, O, O, E-DS, O, O, O, O, O, O, O, O, ...",cd715fd5-7256-4ad2-8834-fb042734d86a,1,ADNI
85,In ADNI 1 participants with available genotyp...,"[O, O, E-DS, O, O, O, O, O, O, O, O, O, O, O, ...",cd715fd5-7256-4ad2-8834-fb042734d86a,1,ADNI
91,"Specifically, using Cox proportional hazard m...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",cd715fd5-7256-4ad2-8834-fb042734d86a,1,ADNI
121,When the findings were released in December 2...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-D...",07d9c326-cd53-4f6c-acae-ad941becc26f,1,Trends in International Mathematics and Scienc...


In [10]:
(dfs["Label"]==1).mean()

0.007922803890869648